In [15]:
import os
import numpy as np
import faiss
import ollama
from tqdm import tqdm
import pickle

In [16]:


# 步骤2: 配置常量
EMBEDDING_MODEL = "nomic-embed-text"
VECTOR_STORE_DIR = "./vector_stores"
INDEX_NAME = "my_documents"
MODEL_NAME = "llama3.2:latest" 

# 步骤3: 加载向量存储
def load_vector_store(index_name):
    index_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}.faiss")
    metadata_path = os.path.join(VECTOR_STORE_DIR, f"{index_name}_metadata.pkl")
    
    try:
        index = faiss.read_index(index_path)
        with open(metadata_path, "rb") as f:
            data = pickle.load(f)
            chunks = data['chunks']
            metadata = data['metadata']
        print(f"✅ 成功加载向量存储 | 索引大小: {index.ntotal}")
        return index, chunks, metadata
    except Exception as e:
        print(f"❌ 加载失败: {str(e)}")
        return None, None, None

# 加载向量存储
index, chunks, metadata = load_vector_store(INDEX_NAME)

# 步骤4: 定义查询函数
def query_documents(question, index, chunks, metadata, model_name=MODEL_NAME, k=3):
    """查询文档并获取回答"""
    try:
        # 获取问题嵌入
        response = ollama.embeddings(model=EMBEDDING_MODEL, prompt=question)
        query_embedding = np.array([response['embedding']], dtype=np.float32)
        
        # 搜索相似内容
        distances, indices = index.search(query_embedding, k)
        
        # 获取相关文本块
        context_chunks = [chunks[i] for i in indices[0]]
        context_metadata = [metadata[i] for i in indices[0]]
        
        # 构造上下文
        context = "\n\n".join([
            f"来源: {meta['filename']} 第{meta['page']}页\n内容: {chunk}"
            for chunk, meta in zip(context_chunks, context_metadata)
        ])
        
        # 构造提示
        prompt = f"""
        基于以下上下文信息回答问题：
        {context}
        
        问题: {question}
        回答:
        """
        
        # 获取回答
        response = ollama.chat(
            model=model_name,
            messages=[{'role': 'user', 'content': prompt}]
        )
        
        return response['message']['content'], context_metadata
        
    except Exception as e:
        return f"查询失败: {str(e)}", []

# 步骤5: 交互式文档查询
print("🚀 文档助手已启动! 输入 'exit' 退出")

✅ 成功加载向量存储 | 索引大小: 202
🚀 文档助手已启动! 输入 'exit' 退出


In [17]:
question = "what is time llm"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
TIME-LLM是指通过重programming大型语言模型（Large Language Model）来适应时间系列预测任务的框架。它利用了自然语言指导（Prompt-as-Prefix）和多头关联层（Multi-head Attention Layer）等技术，能够有效地将时间序列数据转换为可以被语言模型理解的形式，从而实现时间系列预测。

📚 来源:
1. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第6页
2. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第9页
3. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第5页


In [18]:
question = "time llm vs tab pfn, what is the difference"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
根据提供的文本信息，TabPFN（Transformer for Tabular Prediction）和TIME-LLM（Time Series Forecasting by Reprogramming Large Language Models）的区别在于：

1. 类型：TabPFN是用于解决小尺寸表格分类问题的Transformer模型，而TIME-LLM是用于时间系列预测的 Transformer模型。
2. 应用场景：TabPFN主要用于解决小尺寸表格分类问题，特别是那些没有缺失特征的情况下；TIME-LLM则用于时间系列预测和重现。
3. 性能：TIME-LLM在时间系列预测方面表现出优于其他方法的巨大优势，包括在少量shot和零个-shot场景中。TabPFN虽然对小尺寸表格分类问题有很好的性能，但在长期预测中没有 TIME-LLM 相同的优势。

总的来说，TIME-LLM是专门设计用于时间系列预测的Transformer模型，而TabPFN则是一个更广泛应用于表格分类的问题的Transformer模型。

📚 来源:
1. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第22页
2. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第33页
3. TIME-LLM- TIME SERIES FORECASTING BY REPROGRAMMING LARGE LANGUAGE MODELS.pdf - 第6页


In [19]:
question = "how to compare results from times llm and tabpfn"
answer, sources = query_documents(question, index, chunks, metadata)

print(f"🤖 助手回答:\n{answer}\n")
print("📚 来源:")
for i, source in enumerate(sources):
    print(f"{i+1}. {source['filename']} - 第{source['page']}页")

🤖 助手回答:
根据 TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf 的内容，我们可以看到以下几点关于时间比较的信息：

1. TabPFN 在计算机上运行速度非常快，平均每 4.4 秒就可以处理一个数据集，这远比其他基线方法快得多。
2. TabPFN 的总体时间（包括调整和预测）大于或等于其他基线方法。
3. tabpfn 和 other baselines 的 time taken for combined fitting, tuning and prediction 分别在 Table 2 中。

为了比较结果，从这些信息中我们可以看到 TabPFN 在运行速度方面是其他基线方法的明显优势。

📚 来源:
1. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第22页
2. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第9页
3. TabPFN- A Transformer That Solves Small Tabular Classification Problems in a Second.pdf - 第33页
